In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

seed_text = "bien y tu?","muy mal", "Soy una tienda online"

encoded_input = tokenizer.encode(seed_text, return_tensors='pt')

generated_text = model.generate(encoded_input)

output_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

print(output_text)

In [ ]:
import telebot 

bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')
@bot.message_handler(commands=['start', 'help'])

def send_welcome(message):
    bot.reply_to(message, "Hola, en que te puedo ayudarte?")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    seed_text = message.text
    encoded_input = tokenizer.encode(seed_text, return_tensors='pt')    
    generated_text = model.generate(encoded_input)
    output_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    bot.reply_to(message, output_text)

bot.polling()

In [ ]:

import telebot
import transformers
import torch

# Inicializar el bot de Telegram
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')

# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):
    # Extraer la pregunta del mensaje
    question = message.text
    #extraer texto de un archivo de texto
    with open('texto.txt', 'r') as file:
        text = file.read().replace('\n', '')

    #el tokenizador acepta archivo mas largo de 512
    if len(text) > 512:
        text = text[-512:]

    inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
    output = model(**inputs)

    # Obtener la respuesta
    answer_start_scores, answer_end_scores = output[:2]
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end])

    #manejar errores de respuesta
    if answer == '[CLS]':
        answer = 'No entiendo tu pregunta'

    #manejar problemas de respuesta
    if answer == '[SEP]':
        answer = 'No entiendo tu pregunta'
    

    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)

    #guardar las preguntas en texto.txt
    with open('texto.txt', 'a') as file:
        file.write(question + ' ' + answer + ' ')
    
#el programa no se detiene nunca
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)




In [ ]:
import telebot
import transformers
import torch
import time 

# Inicializar el bot de Telegram
bot = telebot.TeleBot('5946891713:AAEqH_b0lL4d26_HfX73EvV1Ny6fsh1jhNM')

# Cargar el modelo BERT pre-entrenado
model = transformers.BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', max_length=10000)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Manejar mensajes de pregunta de Telegram
@bot.message_handler(func=lambda message: True)
def answer_question(message):
    # Extraer la pregunta del mensaje
    question = message.text
    #extraer texto de un archivo de texto
    with open('texto.txt', 'r') as file:
        text = file.read().replace('\n', '')

    #el tokenizador acepta archivo mas largo de 512
    if len(text) > 512:
        text = text[-512:]

    inputs = tokenizer.encode_plus(question, text, return_tensors='pt', add_special_tokens=True)
    output = model(**inputs)

    # Obtener la respuesta
    answer_start_scores, answer_end_scores = output[:2]
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end])
    
    # Enviar la respuesta al usuario
    bot.reply_to(message, answer)

    #guardar las preguntas en texto.txt
    with open('texto.txt', 'a') as file:
        file.write(question + ' ' + answer + ' ')

#el programa no se detiene nunca
while True:
    try:
        bot.polling()
    except Exception:
        time.sleep(15)
